In [1]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip

In [2]:
# Preprocessing
def download_and_analyze(video_url):
  """Downloads and analyzes a video, returning video capture object, FPS, and frame count."""
  # Download video using libraries like urllib
  video = cv2.VideoCapture(video_url)
  fps = video.get(cv2.CAP_PROP_FPS)  #fame per second
  frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
  return video, fps, frame_count

In [3]:
def get_predefined_clips(filepath):
  """Reads pre-determined clip timings from a file (replace with your logic)."""
  clips = []
  with open(filepath, 'r') as f:
    for line in f:
      start, end = line.strip().split(',')
      clips.append((int(start), int(end)))
  return clips

In [4]:
# If clip range not given
def detect_interesting_clips(video, fps, frame_thresh=100):
  """Detects interesting clips based on motion using frame differencing."""
  clips = []
  prev_gray = cv2.cvtColor(video.read()[1], cv2.COLOR_BGR2GRAY)
  while True:
    ret, frame = video.read()
    if not ret:
      break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_delta = cv2.absdiff(prev_gray, gray)
    thresh = cv2.threshold(frame_delta, frame_thresh, 255, cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    if len(cnts) > 0:
      start_frame = video.get(cv2.CAP_PROP_POS_FRAMES) - 1
      end_frame = video.get(cv2.CAP_PROP_POS_FRAMES)
      clips.append((start_frame / fps, end_frame / fps))
    prev_gray = gray
  return clips

In [5]:
# Short Video Creation
def create_short_video(video, clips, music_path, fps=30):
  """Creates a short video with transitions and text overlays."""
  output_video = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (480, 864))  # Adjust resolution if needed
  music = cv2.VideoCapture(music_path)
  font = cv2.FONT_HERSHEY_SIMPLEX

  for i, clip in enumerate(clips):
    start_frame, end_frame = clip
    start_frame = int(start_frame * fps)
    end_frame = int(end_frame * fps)
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    text = f"Clip {i+1}"  # Example text overlay
    text_size, _ = cv2.getTextSize(text, font, 1, 2)
    text_pos = (10, text_size[1] + 10)

    while video.get(cv2.CAP_PROP_POS_FRAMES) <= end_frame:
      ret, frame = video.read()
      if not ret:
        break
      # Resize frame to fit within 9:16 aspect ratio while maintaining content
      frame = resize_and_crop(frame, (480, 864))
      # Add text overlay at specific position
      cv2.putText(frame, text, text_pos, font, 1, (255, 255, 255), 2)

      ret, music_frame = music.read()
      if ret:
        # Add logic to synchronize transitions with music rhythm (e.g., using audio analysis libraries)
        frame = add_transition(frame, music_frame, effect="cut")  # Replace with chosen transition
      else:
        # Handle cases where music ends before video clips
        pass

      cv2.imshow('Output', frame)
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

  output_video.release()
  music.release()


In [6]:
def resize_and_crop(frame, target_size):
  """Resizes and crops a frame to fit within the target size while maintaining content.

  Args:
      frame: The input frame as a NumPy array.
      target_size: A tuple representing the desired width and height of the output frame.

  Returns:
      A resized and cropped frame as a NumPy array.
  """
  frame_height, frame_width, _ = frame.shape
  target_height, target_width = target_size

  # Calculate aspect ratios
  frame_aspect_ratio = frame_width / frame_height
  target_aspect_ratio = target_width / target_height

  # Determine crop based on aspect ratio differences
  if frame_aspect_ratio > target_aspect_ratio:
    # Crop width
    crop_width = int((frame_width - target_width * frame_aspect_ratio) / 2)
    return cv2.resize(frame[:, crop_width:-crop_width], target_size)
  else:
    # Crop height
    crop_height = int((frame_height - target_height / frame_aspect_ratio) / 2)
    return cv2.resize(frame[crop_height:-crop_height, :], target_size)

In [7]:
def add_transition(clip1, clip2, effect="cut", num_frames=10):
  """Applies a transition effect between two video clips.

  Args:
      clip1: The first video clip as a NumPy array.
      clip2: The second video clip as a NumPy array.
      effect: The type of transition effect (e.g., "cut", "dissolve", "wipe").
      num_frames: Number of frames to use for the transition (optional).

  Returns:
      A single video clip with the applied transition between the two input clips.
  """
  if effect == "cut":
    return np.concatenate((clip1, clip2), axis=0)
  elif effect == "dissolve":
    # Implement logic for dissolve transition using alpha blending (example)
    transition_frames = []
    for i in range(num_frames):
      alpha = i / (num_frames - 1)
      frame = cv2.addWeighted(clip1, 1 - alpha, clip2, alpha, 0)
      transition_frames.append(frame)
    return np.concatenate(transition_frames + clip2[num_frames:], axis=0)
  # Add similar logic for other effects like wipe
  else:
    return clip1  # Handle unsupported effects gracefully (optional)


In [8]:
# Finalization
def finalize_and_export(video_path, audio_path, output_path):
  """Combines audio and video, and exports the final short video."""
  # Use libraries like moviepy to combine audio and video streams
  # Export the final video in the desired format (e.g., MP4)
  video_clip = VideoFileClip(video_path).subclip(0, 30)  # Limit video to 30 seconds
  audio_clip = AudioFileClip(audio_path)
  final_clip = video_clip.set_audio(audio_clip)
  final_clip.write_videofile(output_path)




In [9]:
# Main Script
if __name__ == "__main__":
  video_url = "rocket_video.mp4"  # Replace with actual URL or file path
  music_path = "music.mp3"  # Replace with music file path
  clips = [(5, 9), (15, 20), (35, 40)] #get_predefined_clips("clip_timings.txt")  # Or use detect_interesting_clips()

  video, fps, _ = download_and_analyze(video_url)
  create_short_video(video, clips, music_path, fps)
  finalize_and_export("output.mp4", music_path, "final_short_video.mp4")

  video.release()